# Sychronisation de la copie de HAL
1. Identification des notices supprimées depuis la dernière synchro
    voir script Synchro_Premiere_Partie_HAL_DATADCIS



2. Synchronisation de la copie de HAL
    * API en sortie xml-tei pour rechercher les notices nouvelles et les notices modifiées
        * C'est le critère "dateLastIndexed_tdate" qui est préféré à "dateModified_tdate" et à "dateReleased_tdate" pour couvrir les notices créées, les notices modifiées et celles qui sont réindexées pour d'autres raisons.
        * critère : q=dateLastIndexed_tdate:[NOW-1DAY%20TO%20NOW/HOUR]. Explication : [maintenant début de l'heure courante - 1 JOUR (=24h)] JUSQU'A [maintenant début de l'heure courante])
        * pour info : Pour tester avec dates exactes : api.archives-ouvertes.fr/search/INRIA2?q=dateLastIndexed_tdate:["2024-07-03T15:00:00.224Z"%20TO%20"2024-07-04T16:00:00.224Z"]&rows=1&wt=php&fl=docid,uri_s,dateLastIndexed_tdate&sort=docid%20asc&cursorMark=*
        * autres critères : rows=1 et cursorMark pour itérer sur chaque notice du résultat
        * notices
            * enregistrer chaque notice xml-tei (métadonnés de hal) correspondant à chaque docid (il y a un docid par version, donc cela permet d'enregistrer toutes les versions.
            * format du nom du fichier : halID_version.xml (exemple : hal-01044648_v1.xml)
            * dossier de stockage : /data/année/date_du_jour/Notices_Xml-tei_de_Hal
        * fichiers
            * Enregistrer uniquement le fichier PDF associé à la notice (l'objectif étant d'en extraire le full text dans un deuxième temps) dans un dossier correspondant à l'année et un sous-dossier correspondant au jour.
            * Critère d'identification des fichiers dans la notice : existence de balises \<ref\> dans \<edition\>, qui contiennent les liens vers tous les fichiers déposés dans la notice et aussi les liens externalLink
            * format du nom du fichier : " HALID_version_nom_du_fichier.extension du fichier" . exemple : inria-00441254_v2_NEL-splitting.pdf
            * exclusion : tout ce qui n'est pas un fichier PDF (page html ou juste "document" (ex: hal-01000423), tout ce qui est ExternalLink (car on ne connaît pas la licence) et tout ce qui est sous embargo.
            * Embargo : un fichier est créé qui liste les fichiers sous embargo avec la date.
            * dossier de stockage : ../pdf_de_hal/année/date_du_jour
        





## Synchronisation quotidienne : ajout des notices nouvelles ou modifiées

In [20]:
###################################################
## Ajout quotidien de notices nouvelles / modifiées,  et de leurs fichiers
#/!\ Retirer le filtre INRIA2 dans base_url pour la mise en prod!
#/!\ retirer la limite à 300 du compteur à la fin pour la mise en prod!
###################################################

import requests
import xml.etree.ElementTree as ET
import time
import logging
import re
import os
import pandas as pd
from datetime import datetime, date
import shutil
import urllib.parse
import openpyxl



# Obtenir la date actuelle
date_extraction_current = datetime.now().strftime("%Y_%m_%d")
annee_current = datetime.now().strftime("%Y")




## Spécifier le répertoire de log
log_directory = '/data/log/'
## Créer le répertoire s'il n'existe pas
os.makedirs(log_directory, exist_ok=True)

# Construire le nom du fichier de log avec le chemin complet
log_file = os.path.join(log_directory, date_extraction_current + '_log_synchro_part2_hal.txt')

# Configuration du logger
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s', filemode='w')


# with open(log_file, 'a') as f:
#     f.write("Début du fichier de log.\n")

# URL de base de l'API
base_url = "https://api.archives-ouvertes.fr/search/INRIA"

#Critère : tout ce qui a été indexé les dernières 24H
query = "dateLastIndexed_tdate:[NOW-1DAY TO NOW/HOUR]"

# Paramètres de la requête
params = {
    "wt": "xml-tei",
    "rows": 1,
    "sort": "docid asc"
}

# Définition des variables
embargos = set()

# Définition des compteurs
compteur = 0
affichage = 0
compte_embargo = 0
compte_externalLink = 0
compteur_notices_enregistrees = 0
compteur_telechargements = 0

# Déterminer le chemin du dossier DocidDeHAL (au même niveau que le dossier courant)
docid_folder_xml =f"/data/Notices_Xml-tei_de_Hal/{annee_current}/{date_extraction_current}"
docid_folder_pdf = f"/data/pdf_de_hal/{annee_current}/{date_extraction_current}/" 
# Vérifier que le dossier de sortie existe et le créer si nécessaire
os.makedirs(docid_folder_xml, exist_ok=True)
os.makedirs(docid_folder_pdf, exist_ok=True)

# Initialisation du cursorMark
cursor_mark = "*"
previous_cursor_mark = None

logging.info(f"{base_url}?q={query}&wt={params['wt']}&rows={params['rows']}&sort={params['sort']}&cursorMark={cursor_mark}")


# Définition du namespace
namespaces = {"tei": "http://www.tei-c.org/ns/1.0"}

# Lorsque le cursorMark de la réponse est identique à celui de la requête, c'est qu'on a atteint la dernière notice
while cursor_mark != previous_cursor_mark: # tant que le cursorMark est différent du précédent, on récupère la notice
    compteur += 1
    #print(compteur)
    # Mise à jour du cursorMark
    params["cursorMark"] = cursor_mark # cette valeur est d'abord *, puis la valeur du cursorMark de la requête précédente
    #logging.info(f"CursorMark: {cursor_mark}")

    # construction de l'api (cette méthode évite la transformation des espaces et parenthèses en codes qui génèrent une erreur)
    full_url = f"{base_url}?q={query}&wt={params['wt']}&rows={params['rows']}&sort={params['sort']}&cursorMark={params['cursorMark']}"
    
    # Envoi de la requête GET
    response = requests.get(full_url)

    # Vérification du statut de la réponse
    if response.status_code != 200:
        logging.error(f"Erreur: {response.status_code}")
        break

    # Affichez le contenu de la réponse pour le débogage
    #print(f"Response content:\n{response.content.decode('utf-8')}")

    try:
        # Traitement de la réponse XML
        data = ET.fromstring(response.content)
    except ET.ParseError as e:
        logging.error(f"Erreur de parsing: {e}")
        break

    # Recherche du nombre total de résultats
    total_results = data.find('.//tei:measure[@commodity="totalSearchResults"]', namespaces)
    if total_results is not None and affichage == 0:
        total_quantity = total_results.attrib['quantity']
        affichage += 1
        print(f"Nbre de notices indexées ces dernières 24h: {total_quantity}")
        logging.info(f"Nbre de notices indexées ces dernières 24h: {total_quantity}")

    # Récupération du nouveau cursorMark
    next_cursor_mark = data.attrib.get("next")
    # si pas de cursorMark (en principe, il y en a toujours un), on arrête
    if not next_cursor_mark:
        break


    # Récupération du HalId de la publication
    halID_element = data.find('.//tei:idno[@type="halId"]', namespaces)

    # trouver l'uri:
    uri = data.find('.//tei:idno[@type="halUri"]', namespaces)
    # Trouver la version
    current_edition = data.find('.//tei:edition[@type="current"]', namespaces)

    
    # On exécute jusqu'à ce qu'il n'y ait plus de halID dans la réponse
    if halID_element is not None:
        halID_value = halID_element.text
        print(f"hal ID: {halID_value}")
        # logging.info(f"HAL ID: {halID_value}")
    else:
        logging.error("HAL ID not found")
        break
    
    # Vérifier si la version est indiquée
    if current_edition is not None:
        n_value = current_edition.get('n')
        # logging.info(f'version: {n_value}')
    else:
        n_value = ''

    # Mise à jour du cursorMark pour la prochaine itération
    previous_cursor_mark = cursor_mark
    cursor_mark = next_cursor_mark


#########################################################
# Enregistrement de la notice xml-tei dans un fichier XML
#########################################################
    nom_fichier = os.path.join(docid_folder_xml,f'{halID_value}_{n_value}.xml')
    logging.info(f"notice: {halID_value}_{n_value}.xml'")

    compteur_notices_enregistrees += 1

    with open(nom_fichier, "wb") as f:
        f.write(response.content)



##########################################################
# Fonctions de traitement des noms de fichier
#########################################################

    # Fonction pour nettoyer le nom de fichier
    def clean_filename(filename):
        # Supprimer tout ce qui vient après '?' y compris le '?'
        #exemple : fulltext.pdf?sid=hal --> devient fulltext.pdf
        filename = filename.split('?')[0]
        return filename
    
    # Fonction pour vérifier si le nom de fichier est valide
    def is_valid_filename(filename):
        # Vérifier si le nom de fichier se termine par un point suivi de trois lettres
        return bool(re.match(r'.*\.[a-zA-Z]{3}$', filename))
    
    

######################################################################
# Trouver et enregistrer tous les fichiers PDF associés
# appliquer les mêmes règles pour la section "Embargo" située ensuite
######################################################################

    # Repérer la présence de liens vers des fichiers dans la notice
    refs = data.findall('.//tei:edition[@type="current"]/tei:ref', namespaces)

#     # URL spécifique à exclure (se terminant par /document , qui ne pointe pas vers un fichier)
#     exclude_url_suffix = '/document'
    include_file_extension=("pdf", "PDF")


    # Identifier les sous-éléments de chaque "ref" (lien vers des fichiers)

    for ref in refs:
        ref_type = ref.get('type')
        subtype = ref.get('subtype')
        n = ref.get('n')
        target = ref.get('target') # contient l'url du fichier
        date_tag = ref.find('tei:date', namespaces)
        # print(f"ce ref {ref_type} - {subtype} - {n} - {target} - {date_tag} ")

        telecharge = False

        # On ne télécharge que si l'extension est pdf ou PDF
        if target.endswith(include_file_extension):
            
            telecharge = True
            print(f"Target {target} se termine par pdf")


        # identification de la date pour analyse
            if date_tag is not None:
                not_before_date = date_tag.get('notBefore')
                # print(f"not before : {not_before_date}")

                if not_before_date:
                    # Convertir la date au format AAAA-MM-JJ
                    ref_date = datetime.strptime(not_before_date, '%Y-%m-%d').date()
                    #print(ref_date)

                    current_date = date.today()
                    # print(f"date revue: {ref_date} et date du jour : {current_date}")

            # Vérifier que la date n'est pas postérieure à la date actuelle
            if ref_date <= current_date:
                print(f"pas sous embargo")
                embargo = 'False'
                telecharge = 'ok' 
                
            else:
                embargo = 'True'
                if ref_type == 'file':
                    embargomainfile = True
                else:
                    embargomainfile = False
                logging.info(f"embargo à {ref_date} pour {halID_value}")         
                print(f"embargo à {ref_date} pour {halID_value}")         
                # Ajouter les valeurs dans la liste
                if target.endswith(include_file_extension):
                    embargos.add((ref_date, halID_value, embargomainfile, target))
                    print (f" embargo: {ref_date} - {halID_value} - {embargomainfile} - {target}")     
        
            # Extraire le nom de fichier et enregistrer le fichier 
            print(f"telecharge est ok : valeur : {telecharge}")

            # Extraire le nom de fichier et l'extension
            file_name = os.path.basename(target)
            name, ext = os.path.splitext(file_name)

            # Réduire le nom à 100 caractères en incluant l'extension (certains noms de fichier sont trop long et génèrent une interruption du script)
            max_length = 100
            if len(file_name) > max_length:
                truncated_name = name[:max_length - len(ext)]
                file_name = f"{truncated_name}{ext}"

            print(f"nom du fichier éventuellement réduit:{file_name}")

            clean_file_name = clean_filename(file_name) #supprime d'éventuelles url se terminant par ?xxx

            if ref_type == 'file':
                pdf_filename = os.path.join(docid_folder_pdf, f"{halID_value}_{n_value}_main_{clean_file_name}")
                print(f"nom du fichier : {halID_value} subtype : {subtype} main: {pdf_filename}")
            else:
                pdf_filename = os.path.join(docid_folder_pdf, f"{halID_value}_{n_value}_{clean_file_name}")
                print(f"nom du fichier pas 'file' {halID_value} subtype : {subtype} pas main: {pdf_filename}")

        else:
            print(f"Target {target} ne se termine pas par pdf")

            # logging.info(pdf_filename)
#             if is_valid_filename(pdf_filename): # on vérifie que c'est bien un nom de fichier pour éviter les urls qui renvoient sur une page html (ex: elsevier)
#                 try:
#                     response = requests.get(uri)
#                     response.raise_for_status()  # Vérifie si la requête a échoué
#                     with open(pdf_filename, 'wb') as pdf_file:
#                         pdf_file.write(response.content)
#                         compteur_telechargements += 1
#                         # print(f'Téléchargé: {pdf_filename}')
#                         logging.info(f'fichier: {halID_value}_{n_value}_{clean_file_name}')
#                 except requests.exceptions.RequestException as e:
#                     logging.error(f'Échec du téléchargement pour {halID_value}{n_value} - {ref_type} - {date_tag} - {uri}: {e}')
#                 except OSError as e:
#                     logging.error(f'OSError lors de l\'enregistrement du fichier {pdf_filename}: {e}')
#                     continue  # Passer à l'itération suivante en cas d'erreur OSError
#                 except Exception as e:
#                     logging.error(f'Erreur inattendue pour {uri}: {e}')
#                     continue  # Passer à l'itération suivante en cas d'erreur inattendue


#     # Pause pour éviter de surcharger l'API
#     time.sleep(0.1)

    #limite pour tests
    if compteur >= 10:
        break

print(f"Liste des embargos : {embargos}")  # Cette ligne est pour le débogage

# compte_embargo = len(embargos)

# # Chemin du fichier Embargos.xlsx
# embargo_file_path = "/data/log/Embargos.xlsx"

# # Vérifier si le fichier Embargos.xlsx existe déjà
# if os.path.exists(embargo_file_path):
#     # Charger le fichier Excel existant
#     df_existing = pd.read_excel(embargo_file_path)
    
#     # Créer un DataFrame pour les nouvelles données
#     df_new = pd.DataFrame(embargos, columns=['ref_date', 'halID_value','embargomainfile', 'target'])
    
#     # Ajouter les nouvelles données au DataFrame existant
#     df_combined = pd.concat([df_existing, df_new], ignore_index=True)
# else:
#     # Créer un nouveau DataFrame avec les nouvelles données
#     df_combined = pd.DataFrame(embargos, columns=['ref_date', 'halID_value','embargomainfile', 'target'])

# # Enregistrer les données combinées dans Embargos.xlsx
# df_combined.to_excel(embargo_file_path, index=False)



# logging.info(f"notices sous embargo : {compte_embargo}")
# logging.info(f"nb fichiers non enregistrés car externalLink : {compte_externalLink}")
# logging.info(f"nbre notices xml enregistrées : {compteur_notices_enregistrees}")
# logging.info(f"nbre de fichiers enregistrés : {compteur_telechargements}")

# print(f"notices sous embargo : {compte_embargo}")
# print(f"nb fichiers non enregistrés car externalLink : {compte_externalLink}")
# print(f"nbre notices xml enregistrées : {compteur_notices_enregistrees}")
# print(f"nbre de fichiers enregistrés : {compteur_telechargements}")


# logging.info("Synchro 2e partie terminée.")
print("Synchro 2e partie terminée.")



Nbre de notices indexées ces dernières 24h: 2257
hal ID: hal-01001975
Target https://inria.hal.science/hal-01001975v1/document ne se termine pas par pdf
Target https://inria.hal.science/hal-01001975v1/file/art.pdf se termine par pdf
pas sous embargo
telecharge est ok : valeur : ok
nom du fichier éventuellement réduit:art.pdf
nom du fichier : hal-01001975 subtype : author main: /data/pdf_de_hal/2025/2025_06_11/hal-01001975_v1_main_art.pdf
hal ID: hal-01006397
Target https://hal.science/hal-01006397v1/document ne se termine pas par pdf
Target https://hal.science/hal-01006397v1/file/paper.pdf se termine par pdf
pas sous embargo
telecharge est ok : valeur : ok
nom du fichier éventuellement réduit:paper.pdf
nom du fichier : hal-01006397 subtype : author main: /data/pdf_de_hal/2025/2025_06_11/hal-01006397_v1_main_paper.pdf
Target http://hal-ensta.archives-ouvertes.fr/docs/01/00/63/97/PDF/paper.pdf se termine par pdf
pas sous embargo
telecharge est ok : valeur : ok
nom du fichier éventuelleme